## About
DeepNLP Starter 01- RNN and its application for printing "hello world".

- RNN(Recurrent NN) is very special as it takes x as input and produces and output, ensuring the hidden state is being passed to the next state as per the following diagram.

![RNN](RNN.png "RNN")

The series of inputs x1,x2,x3 of each consecutive cell from L to R generates y1,y2,y3 thereby passing hidden states from L to R.

This is useful in handling time series data, language modelling, translation etc task where data is a correlated series of data.

RNNs can be of following types

1. one to many for Image captioning tasks (image as input and caption as sequential input)
2. many to one for sentiment analysis taks (text as input and output - sentiment )
3. many to many for language translation tasks where one language sequential data is fed as input and it generates other language as output.

![RNN2](RNN2.jpg "RNN2")

The above diagram shows the working of RNN where previous state's output(h_t-1) is being concatenated with current state's input x_t and passed to an activation function tanh thereby generating h_t which is being passed to next state.

To use RNN or its complex derivatives in PyTorch, We need to use the following syntax,

```
#output is usually quoted as hidden_size
RNN_cell= nn.RNN(input_size=7, hidden_size=2, batch_first=True)
GRU_cell = nn.GRU(input_size=7, hidden_size=2, batch_first=True)
LSTM_cell = nn.LSTM(input_size=7, hidden_size=2, batch_first=True)

```
- GRU and LSTM have various gates to boost its performance with respect to RNN.


- To implement a general RNN, we should keep the following dimensions of input handy

```
inputs_dim = batch_size, sequence_length, input_size
hidden = cell(inputs, hidden_prev_layer) #dimension - num_layers,batch_size,hidden_size
```


#### 1. Let's teach RNN to say a word - Hello World

1. One hot encode the unique letters in the letters
```
H - [1, 0, 0, 0, 0, 0, 0]
E - [0, 1, 0, 0, 0, 0, 0]
L - [0, 0, 1, 0, 0, 0, 0]
O - [0, 0, 0, 1, 0, 0, 0]
W - [0, 0, 0, 0, 1, 0, 0]
R - [0, 0, 0, 0, 0, 1, 0]
D - [0, 0, 0, 0, 0, 0, 1]
```

2. Feeding each letter to RNN node 

> input_shape to each RNN_cell =  1,1,7
input to first RNN - [[[1, 0, 0, 0, 0, 0, 0]]] #batch_size,seq_len, input_len

> since, we define output_size as 2 i.e hidden_dim
then dimension of h_t to this cell shall be [[[x,x]]] #1,1,2 - num_layers,batchsize, hidden_dim

In [1]:
import torch.nn as nn
import torch
import sys


RNN_Cell = nn.RNN(input_size=7, hidden_size=2, batch_first=True)

inputs = torch.Tensor([[[1, 0, 0, 0, 0, 0, 0]]])
print(inputs.shape)

# initialising an initial hidden state to be passed to first RNN cell
init_hidden_state = torch.randn(1,1,2) # num_layers,batch,hidden_size

out, hidden = RNN_Cell(inputs,init_hidden_state)
print("output of first RNN cell is {}".format(out))
print("hidden state- {}".format(hidden))

#clearly both are same and this shall be passed to next RNN cell.

torch.Size([1, 1, 7])
output of first RNN cell is tensor([[[-0.8517, -0.7473]]], grad_fn=<TransposeBackward1>)
hidden state- tensor([[[-0.8517, -0.7473]]], grad_fn=<StackBackward0>)


#### let's create a many to many RNN stacked where each hidden_state is passed to next cell
#### one for each h,e,l,l,o, w, o,r,l,d

shape = (1,10,7) # batch,seq_len,input_len

#### Also, by increasing batches, We mean we want to forward multiple permutations

e,l,l,o,w,o,r,l,d,h etc
the shape thus becomes

2,10,7

- However, This shall have its imprint on computation cost as it will be slow



#### Teaching RNN to say hello world shall be multi class classification
i.e 
input output pair of each RNN cell
1. h - e, e - l, l- l, l - o, o-w, w-o, o -r , r -l, l-d
2. The loss shall be categorical cross entripy which has softmax implemented in it.

In [2]:
# creating data
# desired input - h,e,l,l,o,w,o,r,l,
# desired output -e,l,l,o,w,o,r,l,d


idx2char = ['h','e','l','o','w','r','d']

input = [0,1,2,2,3,4,3,5,2] # index corresponding to h,e,l,l,o,w,o,r,l,

lookup_table = [[1, 0, 0, 0, 0, 0, 0], # H
                [0, 1, 0, 0, 0, 0, 0], # E
                [0, 0, 1, 0, 0, 0, 0], # L
                [0, 0, 0, 1, 0, 0, 0], # O
                [0, 0, 0, 0, 1, 0, 0], # W
                [0, 0, 0, 0, 0, 1, 0], # R
                [0, 0, 0, 0, 0, 0, 1]] # D

output =  [1,2,2,3,4,3,5,2,6] #-e,l,l,o,w,o,r,l,d



In [3]:
# converting input to one_hot

one_hot_input = [lookup_table[char] for char in input]

print(one_hot_input)

[[1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0], [0, 0, 1, 0, 0, 0, 0]]


In [4]:
#creating tensor for input, output
inputs = torch.Tensor(one_hot_input)
outputs = torch.LongTensor(output) #since, output is not one hot encoded, we will convert it to Long Tensor

In [5]:
inputs,outputs

(tensor([[1., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 1., 0., 0., 0., 0.]]),
 tensor([1, 2, 2, 3, 4, 3, 5, 2, 6]))

In [6]:
#hyper parameters
num_classes = 7
input_size= 7 # one hot size
hidden_size = 7 #to predict one hot
batch_size=1
num_layers =1
sequence_length =1 # feeding just one letter at a time
sequence_length_all = 9



In [7]:
# defining modular class for model

class RNNModel(nn.Module):
    def __init__(self):
        super(RNNModel,self).__init__()
        self.rnn_layer = nn.RNN(input_size=input_size, hidden_size=hidden_size,batch_first=True)

    def forward(self,hidden,x):
        #sys.stdout.write()
        # reshaping input as batch_size,seq_len, input_size
        x = x.view(batch_size, sequence_length,input_size)
        out, hidden = self.rnn_layer(x,hidden)
        #reshaping ouput
        out = out.view(-1,num_classes)
        return hidden, out

    def init_hidden_state(self):
        return torch.zeros(num_layers, batch_size, hidden_size)
        

In [8]:
model = RNNModel()

In [9]:
model

RNNModel(
  (rnn_layer): RNN(7, 7, batch_first=True)
)

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
model = model.to(device)

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

The generated one hot encoded vectors shall be passed via Linear layer to cross entropy loss.

In [ ]:
#let's train


for epoch in range(20000):
    optimizer.zero_grad()
    total_loss = 0
    hidden_state = model.init_hidden_state()    
    for input,label in zip(inputs,outputs):
        input = input.to(device)
        label = label.to(device)
        hidden_state, output = model(hidden_state,input)
        val, idx = output.max(1)
        sys.stdout.write(idx2char[idx.data[0]])

        total_loss+=criterion(output,torch.LongTensor([label]))
        
    print("- Predicted String, Epoch - {}, Loss- {}".format(epoch+1,total_loss))

    total_loss.backward()
    optimizer.step()

elloworld- Predicted String, Epoch - 1, Loss- 5.401774883270264
elloworld- Predicted String, Epoch - 2, Loss- 5.4017744064331055
elloworld- Predicted String, Epoch - 3, Loss- 5.401773929595947
elloworld- Predicted String, Epoch - 4, Loss- 5.401773929595947
elloworld- Predicted String, Epoch - 5, Loss- 5.401773452758789
elloworld- Predicted String, Epoch - 6, Loss- 5.401773452758789
elloworld- Predicted String, Epoch - 7, Loss- 5.401773452758789
elloworld- Predicted String, Epoch - 8, Loss- 5.401773929595947
elloworld- Predicted String, Epoch - 9, Loss- 5.4017744064331055
elloworld- Predicted String, Epoch - 10, Loss- 5.401774883270264
elloworld- Predicted String, Epoch - 11, Loss- 5.401775360107422
elloworld- Predicted String, Epoch - 12, Loss- 5.401776313781738
elloworld- Predicted String, Epoch - 13, Loss- 5.40177583694458
elloworld- Predicted String, Epoch - 14, Loss- 5.40177583694458
elloworld- Predicted String, Epoch - 15, Loss- 5.401773452758789
elloworld- Predicted String, Epoch

elloworld- Predicted String, Epoch - 155, Loss- 5.401679039001465
elloworld- Predicted String, Epoch - 156, Loss- 5.401678562164307
elloworld- Predicted String, Epoch - 157, Loss- 5.401678085327148
elloworld- Predicted String, Epoch - 158, Loss- 5.40167760848999
elloworld- Predicted String, Epoch - 159, Loss- 5.401676177978516
elloworld- Predicted String, Epoch - 160, Loss- 5.401676177978516
elloworld- Predicted String, Epoch - 161, Loss- 5.401675224304199
elloworld- Predicted String, Epoch - 162, Loss- 5.401674747467041
elloworld- Predicted String, Epoch - 163, Loss- 5.401674270629883
elloworld- Predicted String, Epoch - 164, Loss- 5.401673793792725
elloworld- Predicted String, Epoch - 165, Loss- 5.401672840118408
elloworld- Predicted String, Epoch - 166, Loss- 5.40167236328125
elloworld- Predicted String, Epoch - 167, Loss- 5.401671886444092
elloworld- Predicted String, Epoch - 168, Loss- 5.401670932769775
elloworld- Predicted String, Epoch - 169, Loss- 5.401670455932617
elloworld- P

elloworld- Predicted String, Epoch - 321, Loss- 5.401577949523926
elloworld- Predicted String, Epoch - 322, Loss- 5.401577949523926
elloworld- Predicted String, Epoch - 323, Loss- 5.401578426361084
elloworld- Predicted String, Epoch - 324, Loss- 5.401578903198242
elloworld- Predicted String, Epoch - 325, Loss- 5.4015793800354
elloworld- Predicted String, Epoch - 326, Loss- 5.401581764221191
elloworld- Predicted String, Epoch - 327, Loss- 5.401583671569824
elloworld- Predicted String, Epoch - 328, Loss- 5.40158748626709
elloworld- Predicted String, Epoch - 329, Loss- 5.401588439941406
elloworld- Predicted String, Epoch - 330, Loss- 5.40159273147583
elloworld- Predicted String, Epoch - 331, Loss- 5.401588439941406
elloworld- Predicted String, Epoch - 332, Loss- 5.401587009429932
elloworld- Predicted String, Epoch - 333, Loss- 5.4015793800354
elloworld- Predicted String, Epoch - 334, Loss- 5.401573657989502
elloworld- Predicted String, Epoch - 335, Loss- 5.401569366455078
elloworld- Predi

elloworld- Predicted String, Epoch - 485, Loss- 5.401478290557861
elloworld- Predicted String, Epoch - 486, Loss- 5.401478290557861
elloworld- Predicted String, Epoch - 487, Loss- 5.401477813720703
elloworld- Predicted String, Epoch - 488, Loss- 5.401476860046387
elloworld- Predicted String, Epoch - 489, Loss- 5.4014763832092285
elloworld- Predicted String, Epoch - 490, Loss- 5.401474952697754
elloworld- Predicted String, Epoch - 491, Loss- 5.401474952697754
elloworld- Predicted String, Epoch - 492, Loss- 5.4014739990234375
elloworld- Predicted String, Epoch - 493, Loss- 5.401473045349121
elloworld- Predicted String, Epoch - 494, Loss- 5.401473045349121
elloworld- Predicted String, Epoch - 495, Loss- 5.401472091674805
elloworld- Predicted String, Epoch - 496, Loss- 5.4014716148376465
elloworld- Predicted String, Epoch - 497, Loss- 5.40147066116333
elloworld- Predicted String, Epoch - 498, Loss- 5.401470184326172
elloworld- Predicted String, Epoch - 499, Loss- 5.401469707489014
elloworl

elloworld- Predicted String, Epoch - 648, Loss- 5.401398181915283
elloworld- Predicted String, Epoch - 649, Loss- 5.401393413543701
elloworld- Predicted String, Epoch - 650, Loss- 5.401391983032227
elloworld- Predicted String, Epoch - 651, Loss- 5.401386260986328
elloworld- Predicted String, Epoch - 652, Loss- 5.401381015777588
elloworld- Predicted String, Epoch - 653, Loss- 5.4013776779174805
elloworld- Predicted String, Epoch - 654, Loss- 5.401376724243164
elloworld- Predicted String, Epoch - 655, Loss- 5.401377201080322
elloworld- Predicted String, Epoch - 656, Loss- 5.401378631591797
elloworld- Predicted String, Epoch - 657, Loss- 5.40138053894043
elloworld- Predicted String, Epoch - 658, Loss- 5.40138053894043
elloworld- Predicted String, Epoch - 659, Loss- 5.401378631591797
elloworld- Predicted String, Epoch - 660, Loss- 5.401375770568848
elloworld- Predicted String, Epoch - 661, Loss- 5.401373863220215
elloworld- Predicted String, Epoch - 662, Loss- 5.401371955871582
elloworld- 

...

elloworld- Predicted String, Epoch - 16827, Loss- 5.394810199737549
elloworld- Predicted String, Epoch - 16828, Loss- 5.394810199737549
elloworld- Predicted String, Epoch - 16829, Loss- 5.394811153411865
elloworld- Predicted String, Epoch - 16830, Loss- 5.394810199737549
elloworld- Predicted String, Epoch - 16831, Loss- 5.394809246063232
elloworld- Predicted String, Epoch - 16832, Loss- 5.394808292388916
elloworld- Predicted String, Epoch - 16833, Loss- 5.3948073387146
elloworld- Predicted String, Epoch - 16834, Loss- 5.394805908203125
elloworld- Predicted String, Epoch - 16835, Loss- 5.394804954528809
elloworld- Predicted String, Epoch - 16836, Loss- 5.39480447769165
elloworld- Predicted String, Epoch - 16837, Loss- 5.394803524017334
elloworld- Predicted String, Epoch - 16838, Loss- 5.394804000854492
elloworld- Predicted String, Epoch - 16839, Loss- 5.394803524017334
elloworld- Predicted String, Epoch - 16840, Loss- 5.394803524017334
elloworld- Predicted String, Epoch - 16841, Loss- 5

elloworld- Predicted String, Epoch - 16994, Loss- 5.394758701324463
elloworld- Predicted String, Epoch - 16995, Loss- 5.3947577476501465
elloworld- Predicted String, Epoch - 16996, Loss- 5.394758224487305
elloworld- Predicted String, Epoch - 16997, Loss- 5.394757270812988
elloworld- Predicted String, Epoch - 16998, Loss- 5.3947577476501465
elloworld- Predicted String, Epoch - 16999, Loss- 5.39475679397583
elloworld- Predicted String, Epoch - 17000, Loss- 5.394756317138672
elloworld- Predicted String, Epoch - 17001, Loss- 5.394756317138672
elloworld- Predicted String, Epoch - 17002, Loss- 5.394756317138672
elloworld- Predicted String, Epoch - 17003, Loss- 5.3947553634643555
elloworld- Predicted String, Epoch - 17004, Loss- 5.3947553634643555
elloworld- Predicted String, Epoch - 17005, Loss- 5.394754409790039
elloworld- Predicted String, Epoch - 17006, Loss- 5.394754886627197
elloworld- Predicted String, Epoch - 17007, Loss- 5.394754409790039
elloworld- Predicted String, Epoch - 17008, L

elloworld- Predicted String, Epoch - 17162, Loss- 5.394711494445801
elloworld- Predicted String, Epoch - 17163, Loss- 5.394713401794434
elloworld- Predicted String, Epoch - 17164, Loss- 5.394712924957275
elloworld- Predicted String, Epoch - 17165, Loss- 5.394713401794434
elloworld- Predicted String, Epoch - 17166, Loss- 5.394711017608643
elloworld- Predicted String, Epoch - 17167, Loss- 5.394711017608643
elloworld- Predicted String, Epoch - 17168, Loss- 5.394709587097168
elloworld- Predicted String, Epoch - 17169, Loss- 5.394708156585693
elloworld- Predicted String, Epoch - 17170, Loss- 5.394706726074219
elloworld- Predicted String, Epoch - 17171, Loss- 5.394705772399902
elloworld- Predicted String, Epoch - 17172, Loss- 5.394705772399902
elloworld- Predicted String, Epoch - 17173, Loss- 5.394705772399902
elloworld- Predicted String, Epoch - 17174, Loss- 5.394704818725586
elloworld- Predicted String, Epoch - 17175, Loss- 5.394705772399902
elloworld- Predicted String, Epoch - 17176, Loss

elloworld- Predicted String, Epoch - 17328, Loss- 5.394669532775879
elloworld- Predicted String, Epoch - 17329, Loss- 5.394664287567139
elloworld- Predicted String, Epoch - 17330, Loss- 5.394661903381348
elloworld- Predicted String, Epoch - 17331, Loss- 5.394659996032715
elloworld- Predicted String, Epoch - 17332, Loss- 5.394659996032715
elloworld- Predicted String, Epoch - 17333, Loss- 5.394659996032715
elloworld- Predicted String, Epoch - 17334, Loss- 5.394660949707031
elloworld- Predicted String, Epoch - 17335, Loss- 5.394662380218506
elloworld- Predicted String, Epoch - 17336, Loss- 5.394662380218506
elloworld- Predicted String, Epoch - 17337, Loss- 5.394661903381348
elloworld- Predicted String, Epoch - 17338, Loss- 5.394659996032715
elloworld- Predicted String, Epoch - 17339, Loss- 5.39465856552124
elloworld- Predicted String, Epoch - 17340, Loss- 5.394657135009766
elloworld- Predicted String, Epoch - 17341, Loss- 5.394656181335449
elloworld- Predicted String, Epoch - 17342, Loss-

elloworld- Predicted String, Epoch - 17495, Loss- 5.3946213722229
elloworld- Predicted String, Epoch - 17496, Loss- 5.394617080688477
elloworld- Predicted String, Epoch - 17497, Loss- 5.394615650177002
elloworld- Predicted String, Epoch - 17498, Loss- 5.394613265991211
elloworld- Predicted String, Epoch - 17499, Loss- 5.394611835479736
elloworld- Predicted String, Epoch - 17500, Loss- 5.394610404968262
elloworld- Predicted String, Epoch - 17501, Loss- 5.3946099281311035
elloworld- Predicted String, Epoch - 17502, Loss- 5.394609451293945
elloworld- Predicted String, Epoch - 17503, Loss- 5.394610404968262
elloworld- Predicted String, Epoch - 17504, Loss- 5.394610404968262
elloworld- Predicted String, Epoch - 17505, Loss- 5.39461088180542
elloworld- Predicted String, Epoch - 17506, Loss- 5.394611358642578
elloworld- Predicted String, Epoch - 17507, Loss- 5.39461088180542
elloworld- Predicted String, Epoch - 17508, Loss- 5.39461088180542
elloworld- Predicted String, Epoch - 17509, Loss- 5.

### Remarks
We stopped training but we can see as the loss converges - It's converging to print e,l,l,o,w,o,r,l,d

In [ ]:
embedding_size=15
#hyper parameters
num_classes = 7
input_size= 7 # one hot size
hidden_size = 7 #to predict one hot
batch_size=1
num_layers =1
sequence_length =1 # feeding just one letter at a time
sequence_length_all = 9 #feeding entire sequence



class RNNModel_Embedding(nn.Module):
    def __init__(self,num_layers,hidden_size):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size=hidden_size
        self.embedding_layer = nn.Embedding(input_size,embedding_size)
        self.rnn_layer = nn.RNN(input_size=embedding_size, hidden_size=hidden_size,batch_first=True)
        self.fc = nn.Linear(hidden_size,num_classes)

    def forward(self,x):

        #initialise hidden and cell states
        # num_layers* num_directons, batch_size,hidden_size
        h_init = torch.zeros(self.num_layers*1, x.size(0),self.hidden_size)
        embedding = self.embedding_layer(x)
        #reshaping embedding
        embedding = embedding.view(batch_size,sequence_length_all,-1)

        out, h_init = self.rnn_layer(embedding,h_init)

        return self.fc(out.view(-1,num_classes))
        

In [ ]:
model2 = RNNModel_Embedding(num_layers=num_layers,hidden_size=hidden_size)
model2 = model2.to(device)

In [ ]:
model2

In [ ]:
input = [[0,1,2,2,3,4,3,5,2]] # index corresponding to h,e,l,l,o,w,o,r,l,
output =  [1,2,2,3,4,3,5,2,6] #-e,l,l,o,w,o,r,l,d
inputs = torch.LongTensor(input)
labels = torch.LongTensor(output)

In [ ]:
optimizer = torch.optim.Adam(model2.parameters(), lr=0.1)


In [ ]:
#let's train the entire sequence together
idx2char = ['h','e','l','o','w','r','d']

for epoch in range(1000):
    outputs = model2(inputs)
    optimizer.zero_grad()
    loss = criterion(outputs,labels)
    loss.backward()
    optimizer.step()
    val,idx = outputs.max(1)
    idx = idx.data.numpy()

    result = [idx2char[c] for c in idx.squeeze()]
    print("Epoch - {}, Loss - {}".format(epoch+1,loss.item()))
    print("Predicted String- {}".format("".join(result)))


#### Remarks - 
We can see that using RNN with embedding layer followed by a linear layer. The predictions are as desired.

#### Reference

HKUST lectures.